In [1]:
import os,sys,subprocess,glob,cftime,importlib,pickle,itertools
from datetime import datetime
import xarray as xr
import numpy as np
sys.path.append('../')

sys.path.append('../../REA_with_CESM2')
from ensembles.ensemble_GKLT import ensemble_GKLT

from experiment_configuration.c1 import config

%load_ext autoreload
%autoreload 2

In [2]:
'''
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--experiment", type=str)
parser.add_argument("--verbose", action='store_true')
parser.add_argument("--overwrite", action='store_true')
parser.add_argument("--realm", type=str)
parser.add_argument("--h_identifier", type=str)
parser.add_argument("--var_name", type=str)
command_line_arguments = parser.parse_args()
'''

'\nimport argparse\nparser = argparse.ArgumentParser()\nparser.add_argument("--experiment", type=str)\nparser.add_argument("--verbose", action=\'store_true\')\nparser.add_argument("--overwrite", action=\'store_true\')\nparser.add_argument("--realm", type=str)\nparser.add_argument("--h_identifier", type=str)\nparser.add_argument("--var_name", type=str)\ncommand_line_arguments = parser.parse_args()\n'

In [3]:
realm_dict = {
    'atm' : 'atmos',
}

variable_dict = {
    'TREFHT' : 'tas',
    'U500' : 'ua500',
}

In [6]:
overwrite = True
variable = 'TREFHT'
realm = 'atm'
h_identifier = 'h1'
extract_initial = True

In [7]:
for experimetn_identifier in [f"c{i}" for i in range(1,6)] + [f"p{i}" for i in range(1,6)]:
    print(experimetn_identifier)
    # load configuration settings
    experiment_configuration = importlib.import_module(f"experiment_configuration.{experimetn_identifier}")

    # finalize experiment configuration
    from experiment_configuration.experiment import experiment
    exp = experiment(experiment_configuration.config)
    ens = ensemble_GKLT(exp)

    naming_d = {
        "project": 'REA_output',
        "product": exp.product_name,
        "institute": 'NCAR',
        "model": 'CESM2',
        "experiment": f"{exp.initial_conditions_name}-x{exp.experiment_identifier[1]}",
        "time_frequency": "day",
        "realm": realm_dict[realm],
        "variable": variable_dict[variable],
    }

    out_dir = '/'.join([exp.dir_work] + [v for k,v in naming_d.items()])
    out_dir
    
    trajectory_names = sorted([s for s in ens._sim_names if len(s.split('/')) == ens._exp.n_steps])
    trajs = []
    for i,sim_name in enumerate(trajectory_names[:2]):
        out_file_name = f"{out_dir}/{variable}_day_CESM2_{naming_d['experiment']}_ens{str(i+1).zfill(3)}_{exp.initial_condition_fake_year}.nc"
        if os.path.isfile(out_file_name) == False or overwrite:
            todos = [['/'.join(sim_name.split('/')[:step])] for step in range(1,exp.n_steps+1)]
            l = []
            for step in range(1,exp.n_steps+1):
                _sim_name_of_step_ = '/'.join(sim_name.split('/')[:step])
                try:
                    h_file = glob.glob(f"{exp.dir_archive_post}/{_sim_name_of_step_}/{realm}/hist/*{h_identifier}*.nc")[0]
                    with xr.open_mfdataset(h_file) as nc:
                        l.append(nc[variable])
                except:
                    pass
            if len(l) == exp.n_steps:
                x = xr.merge(l)[variable].sortby('time')
                x = x.assign_coords(time=np.array([cftime.DatetimeNoLeap(exp.initial_condition_fake_year,month,day) for month,day in zip(x.time.dt.month.values,x.time.dt.day.values)]))
                x = x.assign_coords(sim=sim_name)
                ds = xr.Dataset({variable:x})
                ds.attrs = nc.attrs
                ds.attrs['simulation_name'] = sim_name
                ds.attrs['initial_condition'] = exp.initial_conditions[i]
                ds.attrs['initial_condition_year'] = exp.initial_conditions[i].split('/')[-1].split('-')[0]
                ds.attrs['compset'] = exp.compset
                ds.attrs['readme'] = "https://github.com/peterpeterp/REA_heat_wEU_JJA"
                ds['time'].attrs['comment'] = f'This simulation represents the climate state of {exp.initial_conditions_name}. The year in the time axis can be ignored.'

                os.makedirs(out_dir, exist_ok=True)
                ds.to_netcdf(out_file_name)


c1
c2
c3
c4
c5
p1
p2
p3
p4
p5


In [11]:
for experimetn_identifier in ["c1","p1"]:
    print(experimetn_identifier)
    # load configuration settings
    experiment_configuration = importlib.import_module(f"experiment_configuration.{experimetn_identifier}")

    # finalize experiment configuration
    from experiment_configuration.experiment import experiment
    exp = experiment(experiment_configuration.config)

    naming_d = {
        "project": 'REA_output',
        "product": exp.product_name,
        "institute": 'NCAR',
        "model": 'CESM2',
        "experiment": f"{exp.initial_conditions_name}-initial",
        "time_frequency": "day",
        "realm": realm_dict[realm],
        "variable": variable_dict[variable],
    }

    out_dir = '/'.join([exp.dir_work] + [v for k,v in naming_d.items()])

    for i,initial_condition in enumerate(exp.initial_conditions[:3]):
        case_identifier = initial_condition.split('.')[-4] + '_' + initial_condition.split('/')[-1].split('-')[0]
        
        out_file_name = f"{out_dir}/{variable}_day_CESM2_{naming_d['experiment']}_ens{str(i+1).zfill(3)}_{exp.initial_condition_fake_year}.nc"
        if os.path.isfile(out_file_name) == False or overwrite:
            archive_fldr = f"{exp.dir_archive}/GKLT/initial_{exp.initial_conditions_name}/{case_identifier}"
            h_files = glob.glob(f"{archive_fldr}/{realm}/hist/*{h_identifier}*.nc")

            try:
                with xr.open_mfdataset(h_files) as nc:
                    x = nc[variable]
                    initial_condition_year = x.time.dt.year.values[0]
                    fake_year = 2025
                    x = x.assign_coords(time=np.array([cftime.DatetimeNoLeap(exp.initial_condition_fake_year,month,day) for month,day in zip(x.time.dt.month.values,x.time.dt.day.values)]))
                    x = x.assign_coords(sim=case_identifier)
                    ds = xr.Dataset({variable:x})
                    ds.attrs = nc.attrs
                    ds.attrs['initial_condition'] = initial_condition
                    ds.attrs['initial_condition_year'] = initial_condition_year
                    ds.attrs['compset'] = exp.compset
                    ds.attrs['readme'] = "https://github.com/peterpeterp/REA_heat_wEU_JJA"
                    ds['time'].attrs['comment'] = f'This simulation represents the climate state of {exp.initial_conditions_name}. The year in the time axis can be ignored.'
                    os.makedirs(out_dir, exist_ok=True)
                    ds.to_netcdf(out_file_name)
            except:
                pass

c1
p1
